In [1]:
import pandas as pd
import geopandas as gpd
import math

In [2]:
wolf_data = pd.read_csv('../data/Cusack_et_al_Wolf_data_2004_2016.csv')

In [3]:
type(wolf_data)

pandas.core.frame.DataFrame

In [4]:
wolf_data.head()

,Wolf_ID,Pack,Easting,Northing,DateTime,Winter
0,380F,SLOUGH CREEK,557939,4978011,20/01/04 20:01,2003
1,380F,SLOUGH CREEK,561583,4981570,21/01/04 02:01,2003
2,380F,SLOUGH CREEK,557152,4978797,21/01/04 08:01,2003
3,380F,SLOUGH CREEK,559197,4978409,21/01/04 11:00,2003
4,380F,SLOUGH CREEK,559099,4979770,21/01/04 14:01,2003


In [5]:
def lat_long(zone, easting, northing, northernHemisphere=True):
    if not northernHemisphere:
        northing = 10000000 - northing

    a = 6378137
    e = 0.081819191
    e1sq = 0.006739497
    k0 = 0.9996

    arc = northing / k0
    mu = arc / (a * (1 - math.pow(e, 2) / 4.0 - 3 * math.pow(e, 4) / 64.0 - 5 * math.pow(e, 6) / 256.0))

    ei = (1 - math.pow((1 - e * e), (1 / 2.0))) / (1 + math.pow((1 - e * e), (1 / 2.0)))

    ca = 3 * ei / 2 - 27 * math.pow(ei, 3) / 32.0

    cb = 21 * math.pow(ei, 2) / 16 - 55 * math.pow(ei, 4) / 32
    cc = 151 * math.pow(ei, 3) / 96
    cd = 1097 * math.pow(ei, 4) / 512
    phi1 = mu + ca * math.sin(2 * mu) + cb * math.sin(4 * mu) + cc * math.sin(6 * mu) + cd * math.sin(8 * mu)

    n0 = a / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (1 / 2.0))

    r0 = a * (1 - e * e) / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (3 / 2.0))
    fact1 = n0 * math.tan(phi1) / r0

    _a1 = 500000 - easting
    dd0 = _a1 / (n0 * k0)
    fact2 = dd0 * dd0 / 2

    t0 = math.pow(math.tan(phi1), 2)
    Q0 = e1sq * math.pow(math.cos(phi1), 2)
    fact3 = (5 + 3 * t0 + 10 * Q0 - 4 * Q0 * Q0 - 9 * e1sq) * math.pow(dd0, 4) / 24

    fact4 = (61 + 90 * t0 + 298 * Q0 + 45 * t0 * t0 - 252 * e1sq - 3 * Q0 * Q0) * math.pow(dd0, 6) / 720

    lof1 = _a1 / (n0 * k0)
    lof2 = (1 + 2 * t0 + Q0) * math.pow(dd0, 3) / 6.0
    lof3 = (5 - 2 * Q0 + 28 * t0 - 3 * math.pow(Q0, 2) + 8 * e1sq + 24 * math.pow(t0, 2)) * math.pow(dd0, 5) / 120
    _a2 = (lof1 - lof2 + lof3) / math.cos(phi1)
    _a3 = _a2 * 180 / math.pi

    latitude = 180 * (phi1 - fact1 * (fact2 + fact3 + fact4)) / math.pi

    if not northernHemisphere:
        latitude = -latitude

    longitude = ((zone > 0) and (6 * zone - 183.0) or 3.0) - _a3

    return (latitude, longitude)

In [6]:
easting = pd.Series(wolf_data['Easting']).astype(float)
northing = pd.Series(wolf_data['Northing']).astype(float)

In [7]:
lat_long(12, easting.iloc[0], northing.iloc[0])

(44.9531733047184, -110.26547431519953)

In [8]:
coors_full = list()
for x in range (0,81712):
    coors = lat_long(12, easting.iloc[x], northing.iloc[x])
    coors_full.append(coors)

    
    

In [9]:
wolf_data['Coordinates'] = coors_full

In [10]:
wolf_data.tail()

,Wolf_ID,Pack,Easting,Northing,DateTime,Winter,Coordinates
81707,1015M,MOLLIE'S,562336,4938461,8/3/2016 5:25,2015,"(44.59679451651816, -110.21457696846869)"
81708,1015M,MOLLIE'S,558388,4935844,10/3/2016 5:00,2015,"(44.573568021056744, -110.26461395333378)"
81709,1015M,MOLLIE'S,558519,4935916,13/03/16 05:00,2015,"(44.57420551734549, -110.26295598163186)"
81710,1015M,MOLLIE'S,562167,4938638,16/03/16 05:01,2015,"(44.598402447888716, -110.21668473444326)"
81711,1015M,MOLLIE'S,558879,4936177,19/03/16 05:00,2015,"(44.57652564284465, -110.25839231396562)"


In [11]:
wolf_data.to_csv('../data/wolf_data.csv', index = False)
 

PermissionError: [Errno 13] Permission denied: '../data/wolf_data.csv'

In [ ]:
coors_df = pd.DataFrame(coors_full)

In [16]:
wolf_ids = wolf_data['Wolf_ID'].value_counts()
wolf_ids.describe()

count      59.000000
mean     1384.949153
std      1379.428510
min         5.000000
25%       254.000000
50%      1018.000000
75%      1946.500000
max      6616.000000
Name: Wolf_ID, dtype: float64